In [11]:
import json
import re
import random
from nltk.tokenize import word_tokenize
import nltk
import spacy
from tqdm import tqdm

# Download NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Utilizador\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Utilizador\AppData\Roaming\nltk_data...


[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [12]:
# Load spaCy model for better topic detection
try:
    nlp = spacy.load('en_core_web_md')
    print("Loaded spaCy model successfully")
except:
    print("Downloading spaCy model (this may take a moment)...")
    import subprocess
    subprocess.run(["python", "-m", "spacy", "download", "en_core_web_md"])
    nlp = spacy.load('en_core_web_md')

# Define science/health/nutrition related terms
TOPIC_KEYWORDS = {
    'science': [
        'science', 'research', 'experiment', 'theory', 'scientific', 'scientist', 'study', 
        'laboratory', 'evidence', 'hypothesis', 'analysis', 'discovery', 'biology', 'physics', 
        'chemistry', 'astronomy', 'geology', 'technology', 'innovation', 'engineering'
    ],
    'health': [
        'health', 'medical', 'medicine', 'disease', 'doctor', 'patient', 'treatment', 'cure', 
        'symptoms', 'diagnosis', 'therapy', 'wellness', 'illness', 'healthcare', 'hospital', 
        'clinic', 'recovery', 'prevention', 'condition', 'immune', 'surgery', 'mental health',
        'physical', 'wellbeing', 'disease', 'cancer', 'heart', 'brain', 'lungs'
    ],
    'nutrition': [
        'nutrition', 'diet', 'food', 'healthy eating', 'nutrient', 'vitamin', 'mineral', 
        'protein', 'carbohydrate', 'fat', 'calorie', 'metabolism', 'digestion', 'meal', 
        'vegetable', 'fruit', 'meat', 'dairy', 'weight', 'obesity', 'sugar', 'organic', 
        'supplement', 'fiber', 'hydration', 'fasting', 'macronutrient'
    ]
}

# Flatten the keywords list for easier checking
ALL_KEYWORDS = set()
for category_keywords in TOPIC_KEYWORDS.values():
    ALL_KEYWORDS.update(category_keywords)

Loaded spaCy model successfully


In [13]:
def clean_text(text):
    """Clean and normalize text"""
    if not text:
        return ""
    # Replace newlines and multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters except punctuation
    text = re.sub(r'[^\w\s.,!?;:\'"-]', '', text)
    return text.strip()

def count_words(text):
    """Count words in text"""
    if not text:
        return 0
    words = word_tokenize(text)
    return len(words)

def is_relevant_topic(text, question):
    """
    Check if the text is relevant to science, health, or nutrition
    using both keyword matching and spaCy similarity
    """
    if not text:
        return False
    
    # Clean and lowercase the combined text
    combined_text = (text + " " + question).lower()
    
    # Direct keyword matching
    for keyword in ALL_KEYWORDS:
        if keyword.lower() in combined_text:
            return True
    
    # Use spaCy for semantic similarity
    doc = nlp(combined_text)
    
    # Check similarity with topic keywords
    for category, keywords in TOPIC_KEYWORDS.items():
        for keyword in keywords:
            keyword_doc = nlp(keyword)
            # Check if any token in the document is similar to the keyword
            for token in doc:
                if token.vector_norm and keyword_doc[0].vector_norm:  # Check if vectors exist
                    similarity = token.similarity(keyword_doc[0])
                    if similarity > 0.75:  # Threshold for similarity
                        return True
    
    return False

def filter_dataset(input_file, output_file, min_words=100, max_words=120, max_entries=1000):
    """
    Filter JSONL dataset to get entries related to science, health, and nutrition
    with word count between min_words and max_words
    """
    # Read the input file and collect valid entries
    valid_entries = []
    line_count = 0
    
    # First pass: count total lines for progress bar
    with open(input_file, 'r', encoding='utf-8') as f:
        for _ in f:
            line_count += 1
    
    print(f"Processing {line_count} entries from {input_file}...")
    
    # Second pass: process entries
    with open(input_file, 'r', encoding='utf-8') as f:
        for line in tqdm(f, total=line_count, desc="Filtering entries"):
            try:
                entry = json.loads(line)
                
                # Clean the text
                answer = clean_text(entry.get('answer', ''))
                question = clean_text(entry.get('question', ''))
                
                # Check word count
                word_count = count_words(answer)
                
                # Check if the entry meets our criteria
                if (min_words <= word_count <= max_words and 
                    is_relevant_topic(answer, question)):
                    
                    # Add the entry along with topic and word count info
                    valid_entries.append({
                        'question': question,
                        'answer': answer,
                        'word_count': word_count
                    })
                    
                    # Print progress periodically
                    if len(valid_entries) % 100 == 0:
                        print(f"Found {len(valid_entries)} valid entries so far")
                        
            except json.JSONDecodeError:
                continue
            except Exception as e:
                print(f"Error processing entry: {e}")
                continue
    
    print(f"Found {len(valid_entries)} entries matching criteria")
    
    # If we have more entries than needed, randomly sample
    if len(valid_entries) > max_entries:
        print(f"Randomly sampling {max_entries} entries from {len(valid_entries)} valid entries")
        valid_entries = random.sample(valid_entries, max_entries)
    
    # Write the filtered entries to the output file
    with open(output_file, 'w', encoding='utf-8') as f:
        for entry in valid_entries:
            f.write(json.dumps(entry) + '\n')
    
    print(f"Saved {len(valid_entries)} entries to {output_file}")
    
    # Print some statistics
    word_counts = [entry['word_count'] for entry in valid_entries]
    if word_counts:
        avg_word_count = sum(word_counts) / len(word_counts)
        print(f"Average word count: {avg_word_count:.2f}")
        print(f"Min word count: {min(word_counts)}")
        print(f"Max word count: {max(word_counts)}")

In [16]:
# Function to convert JSONL to CSV
def convert_to_csv(jsonl_file, csv_file):
    """Convert a JSONL file to CSV format"""
    import csv
    import pandas as pd
    
    # Read JSONL into a list of dictionaries
    data = []
    with open(jsonl_file, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    
    # Convert to DataFrame
    df = pd.DataFrame(data)

    # Removing the question column
    df = df.drop(columns=['question'])
    
    # Save to CSV
    df.to_csv(csv_file, index=False, quoting=csv.QUOTE_ALL)
    print(f"Converted {len(data)} entries to CSV format")
    print(f"Saved to {csv_file}")
    
    return df

In [ ]:
# Example usage in a Jupyter Notebook

# Configuration parameters
input_file = "Quora.jsonl"  # Replace with your input file path
output_file = "filtered_quora.jsonl"  # Replace with desired output file path
min_words = 100  # Minimum word count
max_words = 120  # Maximum word count
max_entries = 1000  # Maximum number of entries to collect

# Run the filtering process
filter_dataset(
    input_file=input_file,
    output_file=output_file,
    min_words=min_words,
    max_words=max_words,
    max_entries=max_entries
)


Processing 56402 entries from Quora.jsonl...


Filtering entries:   5%|▍         | 2619/56402 [00:34<08:13, 108.91it/s]

Found 100 valid entries so far


Filtering entries:   8%|▊         | 4561/56402 [01:02<09:28, 91.20it/s] 

Found 200 valid entries so far


...

Found 2600 valid entries so far


Filtering entries:  98%|█████████▊| 55097/56402 [14:09<00:16, 79.16it/s] 

Found 2700 valid entries so far


Filtering entries: 100%|██████████| 56402/56402 [14:30<00:00, 64.78it/s] 

Found 2760 entries matching criteria
Randomly sampling 1000 entries from 2760 valid entries
Saved 1000 entries to filtered_quora.jsonl
Average word count: 109.92
Min word count: 100
Max word count: 120
Converted 1000 entries to CSV format
Saved to filtered_quora.csv


In [17]:
csv_df = convert_to_csv(output_file, "filtered_quora.csv")

Converted 1000 entries to CSV format
Saved to filtered_quora.csv
